S0 :: Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

S1 :: Load dataframes

In [ ]:
avgConnectivityDf = pd.read_csv("../dataExtracted/avgConnectivity.csv")
rankDf = pd.read_csv("../dataExtracted/rank.csv")

S3 :: Function to filter dataset for relevant iterations and only top 10

In [ ]:
def get_top10(n, from_start= False):
    
    # set start of the slicing
    if from_start:
        start = 0
    else:
        start = n-5
    
    # only keep countries that are in the top10 at any point
    for col in avgConnectivityDf.columns:
        if min(rankDf[col].values) > 10:
            rankDf.drop(col, axis= 1, inplace= True)
            avgConnectivityDf.drop(col, axis= 1, inplace= True)

    # only get data for chosen period
    idx = list(range(start, n))
    conn_df = pd.DataFrame(avgConnectivityDf.loc[idx])

    # sort by the last relevant iteration so that the legend is in order
    return conn_df.T.sort_values(by= n-1, axis= 0, ascending= False)

S4 :: Map each country to a specific color

In [ ]:
columns = avgConnectivityDf.columns
colors = sns.color_palette("turbo", len(columns))

col_dict = {columns[i] : colors[i] for i in range(len(columns))}

S5 :: Interaction !!

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(figsize= (10, 7))

def update_plot(n, from_start= True):

    # get entries    
    conn_df = get_top10(n, from_start)
    
    # set start
    if from_start:
        start = 0
    else:
        start = n-5

    ax.clear()
    ax.grid(alpha= 0.3)
    
    # keep track of minimum connectivity to adjust graph lims
    min_pos_conn = np.inf

    # plot lines
    idx = list(range(start, n))
    for i, (_, r) in enumerate(conn_df.iterrows()):

        # use top10 of final iteration to label countries
        if i < 10:

            # update min conn if needed
            min_conn_row = r.values.min()
            if (min_conn_row <= min_pos_conn):
                min_pos_conn = min_conn_row

            l = f"{i+1}. {r.name}" 
        
        # no label for countries outside of the top10
        else:
            l = None

        ax.plot(list(range(len(idx))), r.values, marker= 'o', label= l, c= col_dict[r.name])
    
    ax.set_xticks(list(range(len(idx))), idx)
    ax.set_ylim(bottom= min_pos_conn-1)  
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    ax.set_ylabel("Connectivity")
    ax.set_xlabel("Iteration")
    ax.set_title(f"Top 10 countries by connectivity.\nFrom iteration {start} to {n-1}.")
    
    fig.tight_layout()
    fig.canvas.draw_idle()


n_slider = widgets.IntSlider(value= 5, min= 5, max= len(avgConnectivityDf)-1, step= 1, description= 'Iteration window')

interact(update_plot, n= n_slider)
plt.show()